In [61]:
import pandas as pd

In [62]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [63]:
df.shape

(891, 12)

In [64]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [65]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [66]:
df.loc[:,['Survived','Pclass','Sex','Embarked']]

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S
...,...,...,...,...
886,0,2,male,S
887,1,1,female,S
888,0,3,female,S
889,1,1,male,C


In [67]:
#This removes the not null rows in the dataframe
df = df.loc[df.Embarked.notna(),['Survived','Pclass','Sex','Embarked']]

In [68]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [69]:
df.shape

(889, 4)

In [70]:
X = df.loc[:,['Pclass']]

In [71]:
y = df.Survived

In [72]:
X.shape

(889, 1)

In [73]:
y.shape

(889,)

In [74]:
from sklearn.linear_model import LogisticRegression

In [75]:
logreg = LogisticRegression(solver = 'lbfgs')

In [76]:
from sklearn.model_selection import cross_val_score

In [77]:
cross_val_score(logreg, X, y, cv = 5, scoring='accuracy').mean()

0.6783406335301212

In [78]:
y.value_counts()

0    549
1    340
Name: Survived, dtype: int64

In [79]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [80]:
ohe.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [81]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [82]:
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [83]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [84]:
X = df.drop('Survived', axis = 'columns')

In [85]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [86]:
from sklearn.compose import make_column_transformer

In [90]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']), remainder='passthrough')

In [91]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [92]:
from sklearn.pipeline import make_pipeline

In [93]:
pipe = make_pipeline(column_trans, logreg)

In [94]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [100]:
X_new = X.sample(5, random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [101]:
pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['Sex', 'Embarked'])],
                      

In [102]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0])